# ML Zoomcamp Homework 2

## Preparation

In [1]:
#getting the data
#!wget -O ../data/housing.csv https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

--2022-09-19 11:10:18--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘../data/housing.csv’

../data/housing.csv 100%[===================>]   1.36M  --.-KB/s    in 0.02s   

2022-09-19 11:10:18 (68.4 MB/s) - ‘../data/housing.csv’ saved [1423529/1423529]



In [2]:
# listing the data folder
!ls ../data

car-prices.csv	housing.csv


## Importing

In [3]:
import numpy  as np
import pandas as pd

## Loading Data

In [4]:
df = pd.read_csv('../data/housing.csv')

In [5]:
df.head(2)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY


In [6]:
# dropping useless cols
df = df.drop('ocean_proximity', axis = 1)
df.head(1)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0


## Question 1
Find a feature with missing values. How many missing values does it have?

In [7]:
result = df.isna().sum().sort_values(ascending=False)
result

total_bedrooms        207
longitude               0
latitude                0
housing_median_age      0
total_rooms             0
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

In [8]:
print(f'The total missing value is {result.iloc[0]}')

The total missing value is 207


## Question 2
What's the median (50% percentile) for variable 'population'?

In [9]:
print(f"The median for population variable is {df['population'].median()}")

The median for population variable is 1166.0


In [10]:
#shuffle the initial dataset, use seed 42.
n = len(df)

n_val = int(n * 0.2)
n_test = int(n * 0.2)
n_train = n - n_val - n_test

In [11]:
n_val, n_test, n_train

(4128, 4128, 12384)

In [12]:
n_val/n, n_test/n, n_train/n

(0.2, 0.2, 0.6)

In [13]:
idx = np.arange(n)

In [14]:
np.random.seed(42)
np.random.shuffle(idx)

In [15]:
df_train = df.iloc[idx[:n_train]]
df_val = df.iloc[idx[n_train:n_train+n_val]]
df_test = df.iloc[idx[n_train+n_val:]]

In [16]:
df_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,47700.0
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,45800.0
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,500001.0
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,218600.0
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,278000.0


In [17]:
len(df_train), len(df_val), len(df_test)

(12384, 4128, 4128)

In [18]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [19]:
y_train = np.log1p(df_train.median_house_value.values)
y_val = np.log1p(df_val.median_house_value.values)
y_test = np.log1p(df_test.median_house_value.values)

In [20]:
del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

## Question 3

In [21]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]

In [22]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [23]:
train_total_bedrooms_mean = df_train['total_bedrooms'].mean()
train_total_bedrooms_mean

533.4803317730147

In [24]:
df_train.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
dtype: int64

In [25]:
df_train.shape

(12384, 8)

In [26]:
X_train_filled_mean = df_train.fillna(train_total_bedrooms_mean).values

In [27]:
X_train_filled_zero = df_train.fillna(0).values

In [28]:
w0, w = train_linear_regression(X_train_filled_mean, y_train)

y_pred = w0 + df_val.dot(w)
round(rmse(y_val, y_pred), 2)

0.33

In [29]:
w0, w = train_linear_regression(X_train_filled_zero, y_train)

y_pred = w0 + df_val.dot(w)
round(rmse(y_val, y_pred), 2)

0.33

## Question 4
Training a regularized Linear Regresion

In [30]:
def train_linear_regression_reg(X, y, r=0.001):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX = XTX + r * np.eye(XTX.shape[0])

    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]

In [31]:
X_train = df_train.fillna(0).values
X_val= df_val.values
X_test= df_test.values

In [32]:
for r in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:
    w0, w = train_linear_regression_reg(X_train, y_train, r=r)
    y_pred = w0 + X_val.dot(w)
    score = rmse(y_val, y_pred)
    
    print('r: ',r, ', w0: ', w0, ', score: ',  round(score,4))

r:  0 , w0:  -11.68697524190301 , score:  0.3295
r:  1e-06 , w0:  -11.686959176238403 , score:  0.3295
r:  0.0001 , w0:  -11.685368865567042 , score:  0.3295
r:  0.001 , w0:  -11.67093131827874 , score:  0.3295
r:  0.01 , w0:  -11.528493585874251 , score:  0.3295
r:  0.1 , w0:  -10.274500282188933 , score:  0.3297
r:  1 , w0:  -4.920480897861685 , score:  0.3338
r:  5 , w0:  -1.482095745626451 , score:  0.3392
r:  10 , w0:  -0.7899311832454328 , score:  0.3406


## Question 5

In [33]:
scores = []

In [34]:
def train_multiple_seed(dataframe, seed_value):
    n = len(dataframe)

    n_val = int(n * 0.2)
    n_test = int(n * 0.2)
    n_train = n - n_val - n_test
    
    idx = np.arange(n)
    np.random.seed(seed_value)
    np.random.shuffle(idx)
    
    df_train = dataframe.iloc[idx[:n_train]]
    df_val = dataframe.iloc[idx[n_train:n_train+n_val]]
    df_test = dataframe.iloc[idx[n_train+n_val:]]
    
    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)
    
    y_train = np.log1p(df_train.median_house_value.values)
    y_val = np.log1p(df_val.median_house_value.values)
    y_test = np.log1p(df_test.median_house_value.values)
    del df_train['median_house_value']
    del df_val['median_house_value']
    del df_test['median_house_value']
    
    X_train = df_train.fillna(0).values
    X_val= df_val.fillna(0).values
    X_test= df_test.fillna(0).values
    
    w0, w = train_linear_regression(X_train, y_train)

    y_pred = w0 + X_val.dot(w)
    score = rmse(y_val, y_pred)
    
    scores.append(score)

In [35]:
for random_seed in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:
    train_multiple_seed(df.copy(), random_seed)

In [36]:
scores

[0.3388430480531664,
 0.3362387255956247,
 0.33209123188322504,
 0.3405153609037416,
 0.33890240665720883,
 0.34348667257165516,
 0.345198095309953,
 0.3395989927407029,
 0.34662308731919184,
 0.3365926124179151]

In [37]:
scores_std = np.std(scores)
round(scores_std, 3)

0.004

## Question 6

In [38]:
def train_multiple_seed(dataframe, seed_value):
    n = len(dataframe)

    n_val = int(n * 0.2)
    n_test = int(n * 0.2)
    n_train = n - n_val - n_test
    
    idx = np.arange(n)
    np.random.seed(seed_value)
    np.random.shuffle(idx)
    
    df_train = dataframe.iloc[idx[:n_train]]
    df_val = dataframe.iloc[idx[n_train:n_train+n_val]]
    df_test = dataframe.iloc[idx[n_train+n_val:]]
    
    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)
    
    df_train = pd.concat([df_train, df_val]).reset_index(drop=True)
    
    y_train = np.log1p(df_train.median_house_value.values)
    y_test = np.log1p(df_test.median_house_value.values)
    
    del df_train['median_house_value']
    del df_test['median_house_value']
    
    X_train = df_train.fillna(0).values
    X_test= df_test.fillna(0).values
    
    
    w0, w = train_linear_regression_reg(X_train, y_train)

    y_pred = w0 + X_test.dot(w)
    score = rmse(y_test, y_pred)
    
    return score

In [39]:
round(train_multiple_seed(df.copy(), 9),2)

0.35